<a href="https://colab.research.google.com/github/kacper-dobrow/tensorflow-course/blob/main/05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
fhvefovhefhoh